In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests5

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2

def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


# a  = .25, k= 4, something not right?

In [3]:
a = 0.25
num_params=4
num_sims = 100

In [4]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,biascorrect=True)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.93 & 0.93 & 0.93 & 0.93 & 0.95 & 0.99   \\
Model 1 & 0.06 & 0.06 & 0.06 & 0.06 & 0.05 & 0.01   \\
Model 2 & 0.01 & 0.01 & 0.01 & 0.01 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.93, 0.06, 0.01]), array([0.93, 0.06, 0.01]), array([0.93, 0.06, 0.01]), array([0.93, 0.06, 0.01]), array([0.95, 0.05, 0.  ]), array([0.99, 0.01, 0.  ]), 0.9992189442955146, 5.033986128440742, 5.318903833827697)


In [5]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,biascorrect=True)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.97 & 0.97 & 0.96 & 0.93 & 0.96 & 0.98   \\
Model 1 & 0.02 & 0.02 & 0.03 & 0.06 & 0.03 & 0.02   \\
Model 2 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.97, 0.02, 0.01]), array([0.97, 0.02, 0.01]), array([0.96, 0.03, 0.01]), array([0.93, 0.06, 0.01]), array([0.96, 0.03, 0.01]), array([0.98, 0.02, 0.  ]), 2.551466879090898, 6.3985866150530875, 6.925414175045732)


# a  = .25, k= 9

In [6]:
a = 0.25
num_params=9
num_sims = 100

In [7]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,biascorrect=True)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.97 & 0.97 & 0.96 & 0.96 & 0.94 & 0.97   \\
Model 1 & 0.02 & 0.02 & 0.03 & 0.03 & 0.04 & 0.02   \\
Model 2 & 0.01 & 0.01 & 0.01 & 0.01 & 0.02 & 0.01   \\
\hline
\end{tabular}
(array([0.97, 0.02, 0.01]), array([0.97, 0.02, 0.01]), array([0.96, 0.03, 0.01]), array([0.96, 0.03, 0.01]), array([0.94, 0.04, 0.02]), array([0.97, 0.02, 0.01]), -2.6286844313380833, 6.248050023106325, 5.788515921143415)


In [8]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,biascorrect=True)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 0.99 & 1.00 & 0.98 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.01 & 0.00 & 0.02 & 0.00   \\
\hline
\end{tabular}
(array([1., 0., 0.]), array([1., 0., 0.]), array([0.99, 0.  , 0.01]), array([1., 0., 0.]), array([0.98, 0.  , 0.02]), array([1., 0., 0.]), -3.3548276921089903, 6.855991753854028, 7.756547808097296)


In [9]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=100, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,biascorrect=True)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.97 & 0.97 & 0.97 & 0.97 & 0.96 & 0.97   \\
Model 1 & 0.03 & 0.03 & 0.03 & 0.03 & 0.03 & 0.03   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.97, 0.03, 0.  ]), array([0.97, 0.03, 0.  ]), array([0.97, 0.03, 0.  ]), array([0.97, 0.03, 0.  ]), array([0.96, 0.03, 0.01]), array([0.97, 0.03, 0.  ]), -3.4116335778216187, 4.153960249587808, 4.271578003223617)


# a  = 0, k= 9

In [10]:
a = 0.00 
num_params=9
num_sims = 100

In [11]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,biascorrect=True)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 0.97 & 0.97 & 0.97 & 0.97   \\
Model 1 & 0.00 & 0.00 & 0.03 & 0.03 & 0.03 & 0.03   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([1., 0., 0.]), array([1., 0., 0.]), array([0.97, 0.03, 0.  ]), array([0.97, 0.03, 0.  ]), array([0.97, 0.03, 0.  ]), array([0.97, 0.03, 0.  ]), -3.2012653094965655, 1.9835004745315523, 2.8201514227316626)


In [12]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,biascorrect=True)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 0.97 & 0.98 & 0.97 & 0.98   \\
Model 1 & 0.00 & 0.00 & 0.03 & 0.02 & 0.03 & 0.02   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([1., 0., 0.]), array([1., 0., 0.]), array([0.97, 0.03, 0.  ]), array([0.98, 0.02, 0.  ]), array([0.97, 0.03, 0.  ]), array([0.98, 0.02, 0.  ]), -3.6862565136301426, 2.0644935509238, 2.9744337728649812)


# a  = .25, K = 19

In [13]:
a = 0.25
num_params=19
num_sims = 100

In [14]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,biascorrect=True)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.97 & 0.97 & 0.97 & 0.97 & 0.97 & 0.97   \\
Model 1 & 0.02 & 0.02 & 0.02 & 0.02 & 0.02 & 0.02   \\
Model 2 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01   \\
\hline
\end{tabular}
(array([0.97, 0.02, 0.01]), array([0.97, 0.02, 0.01]), array([0.97, 0.02, 0.01]), array([0.97, 0.02, 0.01]), array([0.97, 0.02, 0.01]), array([0.97, 0.02, 0.01]), -8.085304639794275, 6.346682411216583, 6.671778393390923)


In [15]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,biascorrect=True)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.97 & 0.97 & 0.97 & 0.97 & 0.97 & 0.98   \\
Model 1 & 0.02 & 0.02 & 0.02 & 0.02 & 0.03 & 0.01   \\
Model 2 & 0.01 & 0.01 & 0.01 & 0.01 & 0.00 & 0.01   \\
\hline
\end{tabular}
(array([0.97, 0.02, 0.01]), array([0.97, 0.02, 0.01]), array([0.97, 0.02, 0.01]), array([0.97, 0.02, 0.01]), array([0.97, 0.03, 0.  ]), array([0.98, 0.01, 0.01]), -8.977318208990722, 7.708504025139934, 8.508004062010984)


In [16]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=1000, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,biascorrect=True)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.99 & 0.99 & 0.98 & 0.99 & 0.99 & 1.00   \\
Model 1 & 0.01 & 0.01 & 0.02 & 0.01 & 0.01 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.99, 0.01, 0.  ]), array([0.99, 0.01, 0.  ]), array([0.98, 0.02, 0.  ]), array([0.99, 0.01, 0.  ]), array([0.99, 0.01, 0.  ]), array([1., 0., 0.]), -7.14542627155899, 9.57687272712051, 11.212615018207096)


# a = 0

In [17]:
a = 0.0
num_params=19
num_sims = 100

In [18]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,biascorrect=True)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 1.00 & 1.00 & 0.98 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.01 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.98, 0.01, 0.01]), array([1., 0., 0.]), -8.597440039632533, 3.1539251971801754, 4.176236290604512)


In [19]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,biascorrect=True)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.95 & 0.95 & 0.96 & 0.95 & 0.96 & 0.96   \\
Model 1 & 0.05 & 0.05 & 0.04 & 0.05 & 0.04 & 0.04   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.95, 0.05, 0.  ]), array([0.95, 0.05, 0.  ]), array([0.96, 0.04, 0.  ]), array([0.95, 0.05, 0.  ]), array([0.96, 0.04, 0.  ]), array([0.96, 0.04, 0.  ]), -8.957967441860616, 3.6913251284111963, 4.289800146046247)


# evidence of power

In [20]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9
num_sims = 100

def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

In [21]:
a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,biascorrect=True)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.50 & 0.50 & 0.51 & 0.50 & 0.51 & 0.59   \\
Model 1 & 0.50 & 0.50 & 0.49 & 0.50 & 0.49 & 0.41   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.5, 0.5, 0. ]), array([0.5, 0.5, 0. ]), array([0.51, 0.49, 0.  ]), array([0.5, 0.5, 0. ]), array([0.51, 0.49, 0.  ]), array([0.59, 0.41, 0.  ]), 6.722757721795949, 4.8487622853531045, 5.201575593215698)


In [22]:
a1,a2 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=500, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,biascorrect=True)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.40 & 0.40 & 0.41 & 0.36 & 0.39 & 0.45   \\
Model 1 & 0.60 & 0.60 & 0.59 & 0.64 & 0.61 & 0.55   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.4, 0.6, 0. ]), array([0.4, 0.6, 0. ]), array([0.41, 0.59, 0.  ]), array([0.36, 0.64, 0.  ]), array([0.39, 0.61, 0.  ]), array([0.45, 0.55, 0.  ]), 8.118418219682143, 5.168530864065854, 5.378208081431068)


In [23]:
a1,a2 = 0.00,np.sqrt(1.09-1)
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,biascorrect=True)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.55 & 0.55 & 0.55 & 0.50 & 0.49 & 0.66   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.45 & 0.45 & 0.45 & 0.50 & 0.51 & 0.34   \\
\hline
\end{tabular}
(array([0.55, 0.  , 0.45]), array([0.55, 0.  , 0.45]), array([0.55, 0.  , 0.45]), array([0.5, 0. , 0.5]), array([0.49, 0.  , 0.51]), array([0.66, 0.  , 0.34]), -13.507910785065066, 4.480773602334858, 5.1165672485226885)


In [24]:
a1,a2 =  0.00,np.sqrt(1.09**.5-1)
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=500, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests5.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,biascorrect=True)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.59 & 0.59 & 0.56 & 0.57 & 0.55 & 0.64   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.41 & 0.41 & 0.44 & 0.43 & 0.45 & 0.36   \\
\hline
\end{tabular}
(array([0.59, 0.  , 0.41]), array([0.59, 0.  , 0.41]), array([0.56, 0.  , 0.44]), array([0.57, 0.  , 0.43]), array([0.55, 0.  , 0.45]), array([0.64, 0.  , 0.36]), -12.870971734286325, 4.849686818186333, 5.101388400620218)
